In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import tensorflow as tf

In [2]:
df = pd.read_csv("task_priority_dataset.csv")


In [3]:
df.head()

,title,description,date_tache,heure,rappel_avant_debut,rappel_retard,recurrence,categorie,priorite
0,Task 0,important deadline,2025-05-27,11:07,113,0,daily,personal,1
1,Task 1,reminder to send email,2025-05-25,16:59,29,9,daily,school,3
2,Task 2,submit report,2025-05-27,16:25,48,51,weekly,personal,2
3,Task 3,team sync,2025-05-26,11:13,10,42,daily,personal,3
4,Task 4,update project plan,2025-05-24,12:53,117,11,daily,personal,3


In [4]:
df["date_tache"] = pd.to_datetime(df["date_tache"])
df["days_until_task"] = (df["date_tache"] - pd.Timestamp.today()).dt.days
df["heure_minutes"] = df["heure"].apply(lambda x: int(x.split(":")[0]) * 60 + int(x.split(":")[1]))

In [5]:
df["recurrence"] = LabelEncoder().fit_transform(df["recurrence"])
df["categorie"] = LabelEncoder().fit_transform(df["categorie"])

In [6]:
X = df[["days_until_task", "heure_minutes", "rappel_avant_debut", "rappel_retard", "recurrence", "categorie"]]
y = df["priorite"] - 1  # convertir à [0, 1, 2]

In [7]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [9]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(16, activation="relu"),
    tf.keras.layers.Dense(8, activation="relu"),
    tf.keras.layers.Dense(3, activation="softmax")  # 3 classes
])

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test))

Epoch 1/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 228ms/step - accuracy: 0.4141 - loss: 1.0231 - val_accuracy: 0.5000 - val_loss: 0.9864
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - accuracy: 0.4641 - loss: 1.0021 - val_accuracy: 0.5000 - val_loss: 0.9702
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.5297 - loss: 0.9641 - val_accuracy: 0.5500 - val_loss: 0.9556
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.5445 - loss: 0.9455 - val_accuracy: 0.5500 - val_loss: 0.9432
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.6016 - loss: 0.9243 - val_accuracy: 0.6000 - val_loss: 0.9313
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.5289 - loss: 0.9408 - val_accuracy: 0.5500 - val_loss: 0.9207
Epoch 7/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.5945 - loss: 0.9310 - val_accuracy: 0.6500 - val_loss: 0.9110
Epoch 8/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.5969 - loss: 0.9255 - val_accuracy: 0.7000 - val_loss: 0.9021

In [11]:
model.export("priority_model")

Saved artifact at 'priority_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 6), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  140375977281680: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140375992530896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140375977279952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140375977279568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140375977283408: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140375977283216: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [12]:
converter = tf.lite.TFLiteConverter.from_saved_model("priority_model")
tflite_model = converter.convert()
with open("priority_model.tflite", "wb") as f:
    f.write(tflite_model)

print("✅ Modèle TFLite sauvegardé sous 'priority_model.tflite'")

✅ Modèle TFLite sauvegardé sous 'priority_model.tflite'
